In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import spacy

In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
resp = pd.read_csv("/Users/anthonychen/Desktop/Yr2Q3/hacks/datafest/Data/Full Data/responses.csv")
resp.head()

/var/folders/3g/fzqltvr90_11xv3v1sxb4f2m0000gn/T/ipykernel_13036/730469401.py:1: DtypeWarning: Columns (32,33,34,35,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  resp = pd.read_csv("/Users/anthonychen/Desktop/Yr2Q3/hacks/datafest/Data/Full Data/responses.csv")


,book,release,chapter,page,chapter_number,section_number,review_flag,institution_id,class_id,student_id,...,lrn_option_2,lrn_option_3,lrn_option_4,lrn_option_5,lrn_option_6,lrn_option_7,lrn_option_8,lrn_option_9,lrn_option_10,lrn_option_11
0,College / Advanced Statistics and Data Science...,v5.0-exp2,Chapter 1 - Welcome to Statistics: A Modeling ...,1.3 Doing Statistics with R,1.0,3.0,False,292cff87-3c74-4e94-8622-233afb0427dd,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab,7535ba88-d131-4eee-b392-36f5e11c85ce,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,College / Advanced Statistics and Data Science...,v5.0-exp2,Chapter 1 - Welcome to Statistics: A Modeling ...,1.3 Doing Statistics with R,1.0,3.0,False,292cff87-3c74-4e94-8622-233afb0427dd,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab,1d4a6d2c-0cb1-4d19-9218-b673982e9f2f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,College / Advanced Statistics and Data Science...,v5.0-exp2,Chapter 1 - Welcome to Statistics: A Modeling ...,1.3 Doing Statistics with R,1.0,3.0,False,292cff87-3c74-4e94-8622-233afb0427dd,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab,94a09f74-2d8c-4379-92ad-7fe5b02ec188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,College / Advanced Statistics and Data Science...,v5.0-exp2,Chapter 1 - Welcome to Statistics: A Modeling ...,1.3 Doing Statistics with R,1.0,3.0,False,292cff87-3c74-4e94-8622-233afb0427dd,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab,20a47775-9594-4800-9857-fd09633c561f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,College / Advanced Statistics and Data Science...,v5.0-exp2,Chapter 1 - Welcome to Statistics: A Modeling ...,1.3 Doing Statistics with R,1.0,3.0,False,292cff87-3c74-4e94-8622-233afb0427dd,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab,7ea35ae6-4aee-4c17-b96a-e1ecb27f2b42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
prompts = resp["prompt"].unique()
prompts = pd.DataFrame({"sentences": prompts})
prompts.head()

,sentences
0,# type whatever you want\n# see... blah blah b...
1,"print(""Hello world!"")\n"
2,"sum(1,5,10)\n"
3,# a few basic arithmetic things\n5 + 1\n10 - 3...
4,<strong>When working on the last few pages of ...


In [6]:
def contains_verb(sentence):
    doc = nlp(str(sentence))
    if str(sentence).strip().endswith('?'):
        return True
    if doc[0].text.lower() in ["who", "what", "when", "where", "why", "how", "use"]:
        return True
    if doc[0].pos_ == 'VERB' and doc[0].tag_ in ["VB", "MD"]:  # "VB" is the base form of a verb
        if len(doc) > 1 and not any(token.dep_ == 'nsubj' for token in doc[1:]):
            return True
    return any(token.pos_ == 'VERB' for token in doc)



In [7]:
prompts["is_question"] = prompts["sentences"].apply(contains_verb)

In [8]:
prompts

,sentences,is_question
0,# type whatever you want\n# see... blah blah b...,True
1,"print(""Hello world!"")\n",False
2,"sum(1,5,10)\n",False
3,# a few basic arithmetic things\n5 + 1\n10 - 3...,False
4,<strong>When working on the last few pages of ...,True
...,...,...
1457,# this saves the best-fitting complex model in...,True
1458,<pre><code>\n Analysis of Variance Table (Type...,True
1459,# modify this to generate a random sample of 1...,True
1460,What does the distribution of the summary vari...,True


In [9]:
prompts.iloc[738,0]

'In a histogram, the height of the bars tells you how many people there are with that thumb length. In a jitter plot, what tells you how many people there are with a particular thumb length?'

In [10]:
resp["lrn_type"].unique()

array([nan, 'choicematrix', 'plaintext', 'mcq', 'association',
       'shorttext', 'clozeassociation', 'imageclozeassociation',
       'sortlist', 'formulaV2'], dtype=object)

In [11]:
resp = resp.merge(prompts, left_on="prompt", right_on="sentences")

In [12]:
resp.columns

Index(['book', 'release', 'chapter', 'page', 'chapter_number',
       'section_number', 'review_flag', 'institution_id', 'class_id',
       'student_id', 'item_id', 'item_type', 'response', 'prompt',
       'points_possible', 'points_earned', 'dt_submitted', 'completes_page',
       'attempt', 'user_agent', 'lrn_response_id', 'lrn_activity_reference',
       'lrn_question_reference', 'lrn_question_position', 'lrn_type',
       'lrn_dt_started', 'lrn_dt_saved', 'lrn_status', 'lrn_option_0',
       'lrn_option_1', 'lrn_option_2', 'lrn_option_3', 'lrn_option_4',
       'lrn_option_5', 'lrn_option_6', 'lrn_option_7', 'lrn_option_8',
       'lrn_option_9', 'lrn_option_10', 'lrn_option_11', 'sentences',
       'is_question'],
      dtype='object')

In [13]:
cols = ["chapter_number", "section_number", "page", "student_id", "item_id", "item_type" ,"response","prompt","points_possible", "points_earned", "review_flag", "dt_submitted", "lrn_type", "is_question", "class_id"]
resp = resp[resp['chapter_number'].notna()]
resp = resp[cols]

In [14]:
resp["lrn_type"] = resp["lrn_type"].fillna("code")

In [15]:
resp.drop("item_type", inplace=True, axis=1)

In [16]:
resp.head()

,chapter_number,section_number,page,student_id,item_id,response,prompt,points_possible,points_earned,review_flag,dt_submitted,lrn_type,is_question,class_id
0,1.0,3.0,1.3 Doing Statistics with R,7535ba88-d131-4eee-b392-36f5e11c85ce,ch1-0,# type whatever you want\n# see... blah blah b...,# type whatever you want\n# see... blah blah b...,1.0,1.0,False,2023-04-04T05:42:37Z,code,True,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab
1,1.0,3.0,1.3 Doing Statistics with R,1d4a6d2c-0cb1-4d19-9218-b673982e9f2f,ch1-0,# type whatever you want\n# see... blah blah b...,# type whatever you want\n# see... blah blah b...,1.0,1.0,False,2023-03-30T05:04:23Z,code,True,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab
2,1.0,3.0,1.3 Doing Statistics with R,94a09f74-2d8c-4379-92ad-7fe5b02ec188,ch1-0,# what is 5+5\n# see a lion\n,# type whatever you want\n# see... blah blah b...,1.0,1.0,False,2023-04-04T06:25:59Z,code,True,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab
3,1.0,3.0,1.3 Doing Statistics with R,20a47775-9594-4800-9857-fd09633c561f,ch1-0,# Sample comment\n# Sample commment 2\n,# type whatever you want\n# see... blah blah b...,1.0,1.0,False,2023-04-04T06:23:59Z,code,True,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab
4,1.0,3.0,1.3 Doing Statistics with R,7ea35ae6-4aee-4c17-b96a-e1ecb27f2b42,ch1-0,# type whatever you want\n# I like cows\n,# type whatever you want\n# see... blah blah b...,1.0,1.0,False,2023-04-04T06:35:06Z,code,True,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab


In [17]:
resp.groupby("lrn_type").count()

,chapter_number,section_number,page,student_id,item_id,response,prompt,points_possible,points_earned,review_flag,dt_submitted,is_question,class_id
lrn_type,,,,,,,,,,,,,
association,35677,35677,35677,35677,35677,35677,35677,35677,35677,35677,35677,35677,35677
choicematrix,34276,34276,34276,34276,34276,34251,27517,6667,6667,34276,34276,34276,34276
clozeassociation,6192,6192,6192,6192,6192,6192,3724,6192,6192,6192,6192,6192,6192
code,384729,384729,384729,384729,384729,384527,384729,384729,384729,384729,384729,384729,384729
formulaV2,1290,1290,1290,1290,1290,1285,1290,1289,1289,1290,1290,1290,1290
imageclozeassociation,2384,2384,2384,2384,2384,2384,1178,2384,2384,2384,2384,2384,2384
mcq,828203,828203,828203,828203,828203,828203,828203,795743,795743,828203,828203,828203,828203
plaintext,198875,198875,198875,198875,198875,198227,198875,4862,3608,198875,198875,198875,198875
shorttext,37348,37348,37348,37348,37348,37266,37348,7949,7949,37348,37348,37348,37348


In [18]:
resp[resp["lrn_type"] == "choicematrix"][resp["points_possible"] == 1][["prompt", "is_question"]]

/var/folders/3g/fzqltvr90_11xv3v1sxb4f2m0000gn/T/ipykernel_13036/1110844293.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  resp[resp["lrn_type"] == "choicematrix"][resp["points_possible"] == 1][["prompt", "is_question"]]


,prompt,is_question
62039,Below we have listed some possible model compa...,True
62040,Below we have listed some possible model compa...,True
62041,Below we have listed some possible model compa...,True
62042,Below we have listed some possible model compa...,True
70710,Which of these are values and which are variab...,True
...,...,...
1569536,Which of the following are true?,True
1569537,Which of the following are true?,True
1569538,Which of the following are true?,True
1569539,Which of the following are true?,True


In [19]:
resp[resp["points_earned"].isna()]["lrn_type"].unique()

array(['choicematrix', 'plaintext', 'mcq', 'shorttext', 'formulaV2'],
      dtype=object)

In [20]:
eoc = pd.read_csv("/Users/anthonychen/Desktop/Yr2Q3/hacks/datafest/Data/Full Data/checkpoints_eoc.csv")
pg_views = pd.read_csv("/Users/anthonychen/Desktop/Yr2Q3/hacks/datafest/Data/Full Data/page_views.csv")

/var/folders/3g/fzqltvr90_11xv3v1sxb4f2m0000gn/T/ipykernel_13036/918466594.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  pg_views = pd.read_csv("/Users/anthonychen/Desktop/Yr2Q3/hacks/datafest/Data/Full Data/page_views.csv")


In [21]:
eoc["chapter_number"] = eoc["chapter_number"].astype(float)

resp["student_ch"] = resp["student_id"].astype(str) + resp["chapter_number"].astype(str)

eoc["student_ch"] = eoc["student_id"].astype(str) + eoc["chapter_number"].astype(str)
eoc2 = eoc[["student_ch", "EOC"]]

pg_views['student_ch'] = pg_views['student_id'].astype(str) + pg_views['chapter_number'].astype(str)
pg_views2 = pg_views[["student_ch", "engaged"]]

df = pd.merge(resp, eoc2, on="student_ch", how="left")
df = pd.merge(df, pg_views2, on="student_ch", how="left")
df.head()

,chapter_number,section_number,page,student_id,item_id,response,prompt,points_possible,points_earned,review_flag,dt_submitted,lrn_type,is_question,class_id,student_ch,EOC,engaged
0,1.0,3.0,1.3 Doing Statistics with R,7535ba88-d131-4eee-b392-36f5e11c85ce,ch1-0,# type whatever you want\n# see... blah blah b...,# type whatever you want\n# see... blah blah b...,1.0,1.0,False,2023-04-04T05:42:37Z,code,True,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab,7535ba88-d131-4eee-b392-36f5e11c85ce1.0,0.904762,852960.0
1,1.0,3.0,1.3 Doing Statistics with R,7535ba88-d131-4eee-b392-36f5e11c85ce,ch1-0,# type whatever you want\n# see... blah blah b...,# type whatever you want\n# see... blah blah b...,1.0,1.0,False,2023-04-04T05:42:37Z,code,True,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab,7535ba88-d131-4eee-b392-36f5e11c85ce1.0,0.904762,426013.0
2,1.0,3.0,1.3 Doing Statistics with R,7535ba88-d131-4eee-b392-36f5e11c85ce,ch1-0,# type whatever you want\n# see... blah blah b...,# type whatever you want\n# see... blah blah b...,1.0,1.0,False,2023-04-04T05:42:37Z,code,True,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab,7535ba88-d131-4eee-b392-36f5e11c85ce1.0,0.904762,7672.0
3,1.0,3.0,1.3 Doing Statistics with R,7535ba88-d131-4eee-b392-36f5e11c85ce,ch1-0,# type whatever you want\n# see... blah blah b...,# type whatever you want\n# see... blah blah b...,1.0,1.0,False,2023-04-04T05:42:37Z,code,True,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab,7535ba88-d131-4eee-b392-36f5e11c85ce1.0,0.904762,9541.0
4,1.0,3.0,1.3 Doing Statistics with R,7535ba88-d131-4eee-b392-36f5e11c85ce,ch1-0,# type whatever you want\n# see... blah blah b...,# type whatever you want\n# see... blah blah b...,1.0,1.0,False,2023-04-04T05:42:37Z,code,True,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab,7535ba88-d131-4eee-b392-36f5e11c85ce1.0,0.904762,403739.0


In [23]:
eoc["chapter_number"] = eoc["chapter_number"].astype(float)

resp["class_ch"] = resp["class_id"].astype(str) + resp["chapter_number"].astype(str)
pg_views["class_ch"] = pg_views["class_id"].astype(str) + pg_views["chapter_number"].astype(str)
eoc["class_ch"] = eoc["class_id"].astype(str) + eoc["chapter_number"].astype(str)

eoc3 = eoc[["class_ch", "EOC"]]
pg_views3 = pg_views[["class_ch", "engaged"]]

pg_views4 = pg_views3.groupby("class_ch").agg("mean")
eoc4 = eoc3.groupby("class_ch").agg("mean")

class_df = pd.merge(resp, eoc4, on="class_ch", how="left")
class_df = pd.merge(class_df, pg_views4, on="class_ch", how="left")
class_df.head()

,chapter_number,section_number,page,student_id,item_id,response,prompt,points_possible,points_earned,review_flag,dt_submitted,lrn_type,is_question,class_id,student_ch,class_ch,EOC,engaged
0,1.0,3.0,1.3 Doing Statistics with R,7535ba88-d131-4eee-b392-36f5e11c85ce,ch1-0,# type whatever you want\n# see... blah blah b...,# type whatever you want\n# see... blah blah b...,1.0,1.0,False,2023-04-04T05:42:37Z,code,True,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab,7535ba88-d131-4eee-b392-36f5e11c85ce1.0,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab1.0,0.900963,279720.012469
1,1.0,3.0,1.3 Doing Statistics with R,1d4a6d2c-0cb1-4d19-9218-b673982e9f2f,ch1-0,# type whatever you want\n# see... blah blah b...,# type whatever you want\n# see... blah blah b...,1.0,1.0,False,2023-03-30T05:04:23Z,code,True,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab,1d4a6d2c-0cb1-4d19-9218-b673982e9f2f1.0,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab1.0,0.900963,279720.012469
2,1.0,3.0,1.3 Doing Statistics with R,94a09f74-2d8c-4379-92ad-7fe5b02ec188,ch1-0,# what is 5+5\n# see a lion\n,# type whatever you want\n# see... blah blah b...,1.0,1.0,False,2023-04-04T06:25:59Z,code,True,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab,94a09f74-2d8c-4379-92ad-7fe5b02ec1881.0,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab1.0,0.900963,279720.012469
3,1.0,3.0,1.3 Doing Statistics with R,20a47775-9594-4800-9857-fd09633c561f,ch1-0,# Sample comment\n# Sample commment 2\n,# type whatever you want\n# see... blah blah b...,1.0,1.0,False,2023-04-04T06:23:59Z,code,True,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab,20a47775-9594-4800-9857-fd09633c561f1.0,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab1.0,0.900963,279720.012469
4,1.0,3.0,1.3 Doing Statistics with R,7ea35ae6-4aee-4c17-b96a-e1ecb27f2b42,ch1-0,# type whatever you want\n# I like cows\n,# type whatever you want\n# see... blah blah b...,1.0,1.0,False,2023-04-04T06:35:06Z,code,True,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab,7ea35ae6-4aee-4c17-b96a-e1ecb27f2b421.0,6fbf5a0a-cf5d-4567-89b5-eb5c4a16c4ab1.0,0.900963,279720.012469


In [28]:
df["chapter_section"] = df["chapter_number"].astype(str) + df["section_number"].astype(str)

In [51]:
df.groupby("chapter_number")["engaged"].mean().sort_values(ascending=False)

chapter_number
12.0    442377.758774
3.0     394674.635484
11.0    393691.724405
10.0    357668.379945
4.0     356112.564051
2.0     324971.398135
6.0     312857.719962
8.0     301442.859862
1.0     294917.923004
9.0     293285.427015
14.0    292388.822946
13.0    288733.472709
5.0     281332.492947
15.0    236226.414764
7.0     222480.929945
16.0    182348.811500
Name: engaged, dtype: float64

In [52]:
ch12 = 442377.758774

In [56]:
avg_engagement = (sum(df.groupby("chapter_number")["engaged"].mean().sort_values(ascending=False)) - ch12) / 15

In [57]:
(ch12 - avg_engagement) / avg_engagement

0.463814438593931